In [1]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore') 

In [2]:
!git clone https://github.com/Omar-Nasr/ai_generated_image_classification_defactify_2024

Cloning into 'ai_generated_image_classification_defactify_2024'...
remote: Enumerating objects: 244, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 244 (delta 55), reused 32 (delta 14), pack-reused 156 (from 1)
Receiving objects: 100% (244/244), 247.25 KiB | 542.00 KiB/s, done.
Resolving deltas: 100% (151/151), done.


In [3]:
import sys
sys.path.append("/kaggle/working/ai_generated_image_classification_defactify_2024")

In [4]:
!cd /kaggle/working/ai_generated_image_classification_defactify_2024 && git pull

Already up to date.


In [5]:
from data_handler import Image_Classification_Dataset

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [6]:
!mkdir checkpoints

In [7]:
train_data_dir = "/kaggle/input/ai-image-classification-2/" 
checkpoint_path = "checkpoints/"
val_data_dir = "/kaggle/input/d/omarnasr/ai-image-classification-val-test/"

In [8]:
!curl "https://drive.usercontent.google.com/download?id=1mutiXZZGP9UhtqOLSDspJHX7VNafT2z0&export=download&authuser=0&confirm=t&uuid=85b29666-6ba1-4a29-830c-7bc011d7a176&at=AENtkXb0JST3mpXXQql8s83jTM4M:1730885596175" --output val.zip

curl: /opt/conda/lib/libcurl.so.4: no version information available (required by curl)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  226k  100  226k    0     0  95940      0  0:00:02  0:00:02 --:--:-- 95915


In [9]:
!unzip -P DEFACTIFY-4_IMG_VAL "/kaggle/working/val.zip"

Archive:  /kaggle/working/val.zip
   creating: val_dataset/
  inflating: val_dataset/Readme_Val_data.md  
  inflating: val_dataset/Validation_labels.xlsx  


In [10]:
val_labels_df = pd.read_excel("/kaggle/working/val_dataset/Validation_labels.xlsx")
val_labels_df

,Index,Caption,Label_A,Label_B
0,0,a toilet sits next to a shower an sink,1,3
1,1,A TV sitting on top of a table next to a lapto...,0,0
2,2,Two giraffes eat from a pot attached to a fence.,1,1
3,3,The kitchen has many grill with pots hanging a...,1,4
4,4,some road signs besides a road in the street,1,1
...,...,...,...,...
8995,8995,Three red double decker buses parked side by s...,0,0
8996,8996,A white toilet in a white tiled bathroom.,1,3
8997,8997,A bus is stopping to pick up a passenger.,0,0
8998,8998,Small birds in the grass in the sun,1,4


In [11]:

val_labels = np.array(val_labels_df.iloc[:,2])
val_multiclass_labels = np.array(val_labels_df.iloc[:,3])
val_multiclass_labels

array([3, 0, 1, ..., 0, 4, 2])

In [12]:
from data_handler import Image_Classification_Dataset
from torch.utils.data import DataLoader
val = Image_Classification_Dataset(val_data_dir,task="Binary",val=True,val_labels=val_labels)
test = Image_Classification_Dataset(val_data_dir,task="Binary",val=False,test=True)
test_loader = DataLoader(test,16,num_workers=4)

In [13]:
from parameter_tuning import objective
from functools import partial
from optuna.trial import TrialState
import optuna 
study = optuna.create_study(direction="maximize")
objective = partial(objective, train_data_dir=train_data_dir,checkpoint_path=checkpoint_path,num_epochs=3,val_data_dir=val_data_dir,val_labels = val_multiclass_labels,val=True,task="Multiclass",model_name="swinb",optimizer_name="adopt",batch_sz=32,freeze_number=0,use_fourier=False)
study.optimize(objective, n_trials=5)
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
with open("bestparams","w") as f:
    for key, value in trial.params.items():
        f.write("    {}: {}".format(key, value))


[I 2024-12-03 14:42:40,326] A new study created in memory with name: no-name-470837e2-b0ed-4646-8d00-cb3c9723ca0f
Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth
100%|██████████| 528M/528M [00:02<00:00, 239MB/s]


Epoch 0/2

----------



libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG
libpng warning: iCCP: known incorrect 

Epoch $0 Training Loss: $53976.08322143555 

Epoch $0 Training F1_Score 0.4863095283508301 Best F1: 0.4863095283508301
Epoch $0 Val Loss: $13269.14221572876 

Epoch $0 Val F1_Score 0.40744444727897644 Best F1: 0.40744444727897644
Epoch 1/2

----------



libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 'desc': ICC p

...

Epoch $1 Training Loss: $82343.82464408875 

Epoch $1 Training F1_Score 0.1714523881673813 Best F1: 0.17588889598846436
Epoch $1 Val Loss: $17664.7855758667 

Epoch $1 Val F1_Score 0.17622222006320953 Best F1: 0.17622222006320953
Epoch 2/2

----------



libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect 

Epoch $2 Training Loss: $81387.14896774292 

Epoch $2 Training F1_Score 0.17838095128536224 Best F1: 0.17838095128536224
Epoch $2 Val Loss: $17433.108978271484 

Epoch $2 Val F1_Score 0.1827777773141861 Best F1: 0.1827777773141861
Training complete in 20m 59s

Best F1: 0.1827777773141861 after 3 Epochs


[I 2024-12-03 16:28:04,161] Trial 4 finished with value: 0.1827777773141861 and parameters: {'lr': 2.739918698951285e-10, 'lr2': 0.0044272191588647005}. Best is trial 1 with value: 0.4718888998031616.


Study statistics: 
  Number of finished trials:  5
  Number of pruned trials:  0
  Number of complete trials:  5
Best trial:
  Value:  0.4718888998031616
  Params: 


In [14]:
# from models import train_classifier
# model_adopt_binary,classifier_adopt = train_classifier(train_data_dir,checkpoint_path,num_epochs=3,val_data_dir=val_data_dir,val_labels = val_labels,val=True,use_fourier=False,optimizer_name="adopt",lr=1e-5,fine_tune=False,batch_sz=64,freeze_number=0)

In [15]:
# from models import train_classifier
# model_adopt_multiclass,classifier_adopt_multiclass = train_classifier(train_data_dir,checkpoint_path,num_epochs=5,val_data_dir=val_data_dir,val_labels = val_multiclass_labels,val=True,task="Multiclass",use_fourier=False,optimizer_name="adopt",lr=1e-5,fine_tune=False)

In [16]:
# from models import train_classifier
# model_adam_binary,classifier_adam = train_classifier(train_data_dir,checkpoint_path,num_epochs=3,val_data_dir=val_data_dir,val_labels = val_labels,val=True,use_fourier=False)
# model_adam_multiclass,classifier_adam_multiclass = train_classifier(train_data_dir,checkpoint_path,num_epochs=3,val_data_dir=val_data_dir,val_labels = val_multiclass_labels,val=True,task="Multiclass",use_fourier=False)

In [17]:
from torchvision import models
# import torch
# device = xm.xla_device()
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model_adopt_binary.to(device)
# classifier_adopt.to(device)
# model_adopt_multiclass.to(device)
# classifier_adopt_multiclass.to(device)
# model_adam_multiclass.to(device)
# classifier_adam_multiclass.to(device)
# test_preds_adam_binary = []
# test_preds_adam_multiclass = []
# test_preds_adopt_binary = []
# test_preds_adopt_multiclass = []
# for inputs in test_loader:
#         inputs = inputs.to(device)
#         features = model_adam_multiclass(inputs)
#         outputs = classifier_adam_multiclass(features)
#         _,preds = torch.max(outputs,1)
#         test_preds_adam_multiclass.append(preds.cpu())
# for inputs in test_loader:
#         inputs = inputs.to(device)
#         features = model_adam_binary(inputs)
#         outputs = classifier_adam(features)
#         _,preds = torch.max(outputs,1)
        # test_preds_adam_binary.append(preds.cpu())    
# for inputs in test_loader:
#         inputs = inputs.to(device)
#         features = model_adopt_multiclass(inputs)
#         outputs = classifier_adopt_multiclass(features)
#         _,preds = torch.max(outputs,1)
#         test_preds_adopt_multiclass.append(preds.cpu())
# for inputs in test_loader:
#         inputs = inputs.to(device)
#         features = model_adopt_binary(inputs)
#         outputs = classifier_adopt(features)
#         _,preds = torch.max(outputs,1)
#         test_preds_adopt_binary.append(preds.cpu())    
# test_preds_adopt_binary = np.concatenate(test_preds_adopt_binary)
# test_preds_adopt_multiclass = np.concatenate(test_preds_adopt_multiclass)
# test_preds_adam_binary = np.concatenate(test_preds_adam_binary)
# test_preds_adam_multiclass = np.concatenate(test_preds_adam_multiclass)


In [18]:
# from datasets import load_dataset
# import json
# ds = load_dataset("NasrinImp/Defactify4_Test")
# captions = ds["train"]['caption']
# list = []
# for i,pred in enumerate(test_preds_adam_binary):
#     dict = {}
#     dict["index"]=i
#     dict["caption"]=captions[i]
#     dict["Label_A"]=int(test_preds_adam_binary[i])
#     dict["Label_B"]=int(test_preds_adam_multiclass[i])
#     list.append(dict)
# json_object = json.dumps(list,indent=4)
# with open("answer_adam.json","w") as f:
#     f.write(json_object)

In [19]:
# from datasets import load_dataset
# import json
# ds = load_dataset("NasrinImp/Defactify4_Test")
# captions = ds["train"]['caption']
# list = []
# for i,pred in enumerate(test_preds_adopt_binary):
#     dict = {}
#     dict["index"]=i
#     dict["caption"]=captions[i]
#     dict["Label_A"]=int(test_preds_adopt_binary[i])
#     dict["Label_B"]=int(test_preds_adopt_multiclass[i])
#     list.append(dict)
# json_object = json.dumps(list,indent=4)
# with open("answer_adopt.json","w") as f:
#     f.write(json_object)